In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error as mae
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
import math
from sys import stdout
%matplotlib inline
#  Провести предподготовку данных
data_train = pd.read_csv('C:/Anaconda/train.csv', delimiter=",", index_col='id')
data_test = pd.read_csv('C:/Anaconda/test.csv', delimiter=",", index_col='id')
#первые строки файла каждого датасета
data_train.head()

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
id,,,,,,,
0,02.02.2015,23.7000,26.272,585.200000,749.200000,0.004764,1
1,02.02.2015,23.7180,26.290,578.400000,760.400000,0.004773,1
2,02.02.2015,23.7300,26.230,572.666667,769.666667,0.004765,1
3,02.02.2015,23.7225,26.125,493.750000,774.750000,0.004744,1
4,02.02.2015,23.7540,26.200,488.600000,779.000000,0.004767,1


In [4]:
data_test.head()

,date,Temperature,Humidity,Light,CO2,HumidityRatio
id,,,,,,
0,12.02.2015,22.315000,26.55,461.50,548.000000,0.004425
1,12.02.2015,22.290000,26.55,454.00,546.250000,0.004418
2,12.02.2015,22.323333,26.60,454.00,542.666667,0.004436
3,12.02.2015,22.290000,26.60,454.00,543.333333,0.004427
4,12.02.2015,22.315000,26.60,450.25,551.750000,0.004433


In [5]:
#количество строк и столбцов в каждом датасете
data_train.shape

(12336, 7)

In [6]:
data_test.shape

(8224, 6)

In [9]:
#определим баланс классов
#количество единиц
data_train_count_y1 = data_train[data_train['Occupancy']==1].shape[0]
#количество нулей
data_train_count_y0 = data_train[data_train['Occupancy']==0].shape[0]
print("Количество единиц = {}, количество нулей = {}"\
     .format(data_train_count_y1, data_train_count_y0,\
             data_train_count_y1 + data_train_count_y0))

Количество единиц = 3070, количество нулей = 9266


In [10]:
#построение корреляционной матрицы
data_train.corr()

,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
Temperature,1.000000,-0.049097,0.655458,0.597588,0.323295,0.524851
Humidity,-0.049097,1.000000,0.097879,0.444851,0.927516,0.188871
Light,0.655458,0.097879,1.000000,0.685222,0.338397,0.908617
CO2,0.597588,0.444851,0.685222,1.000000,0.662796,0.711089
HumidityRatio,0.323295,0.927516,0.338397,0.662796,1.000000,0.383737
Occupancy,0.524851,0.188871,0.908617,0.711089,0.383737,1.000000


In [11]:
#проверка на пропущенные значения
if data_train.columns[data_train.isnull().values.any()].tolist() == []:
    print('Нет пропущенных')

Нет пропущенных


In [12]:
#Обучить модель из sklearn
#Разбиение на train и validation set(для обучения и валидации)
from sklearn.model_selection  import train_test_split
train, validation = train_test_split(data_train, test_size=0.2)

In [13]:
#используем метрику - точность
from sklearn.metrics import accuracy_score
arr_name = []
arr_train = []
arr_val = []

cols_x = ['Temperature','Light','CO2','HumidityRatio']   
col_y = 'Occupancy'

def test_classifier(classifier,classifier_name):
    classifier.fit(train[cols_x], train[col_y])
    
    y_train = classifier.predict(train[cols_x])
    
    y_train_acc = accuracy_score(train[col_y], y_train)
    y_val = classifier.predict(validation[cols_x])
    y_val_acc = accuracy_score(validation[col_y], y_val)
    
    arr_name.append(classifier_name)
    arr_train.append(y_train_acc)
    arr_val.append(y_val_acc)
    
    print('Точность для алгоритма {} на обучающей выборке={},\
    на валидационной выборке ={}'\
          .format(classifier_name,\
                  round(y_train_acc, 3),\
                  round(y_val_acc, 3)))
    
    return classifier

In [14]:
#логистическая регрессия
from sklearn.linear_model import LogisticRegression
%time classifier = test_classifier(LogisticRegression(),'LR' )

Точность для алгоритма LR на обучающей выборке=0.986,    на валидационной выборке =0.99
Wall time: 56.8 ms


In [15]:
#заполним пустые количественные медианным значением

data_train = data_train.fillna(data_train.median(axis=0), axis=0)

categorical_columns = [c for c in data_train.columns if data_train[c].dtype.name == 'object']
numerical_columns   = [c for c in data_train.columns if data_train[c].dtype.name != 'object']
#заполним пустые категориальные самым частым значением по признаку
data_describe = data_train.describe(include=[object]) #получение сводной информации по таблице
for c in categorical_columns:
    data_train[c] = data_train[c].fillna(data_describe[c]['top'])  # fillna() - метод для замены отсутствующих значений на числовые
#преобразование в количественные
binary_columns    = [c for c in categorical_columns if data_describe[c]['unique'] == 2] #бинарные
nonbinary_columns = [c for c in categorical_columns if data_describe[c]['unique'] > 2] #небинарные

data_describe = data_train.describe(include=[object])

for c in binary_columns:
    top = data_describe[c]['top']
    top_items = data_train[c] == top
    data_train.loc[top_items, c] = 0
    data_train.loc[np.logical_not(top_items), c] = 1
       

data_nonbinary = pd.get_dummies(data_train[nonbinary_columns])

#нормализация количественных признаков

data_numerical = data_train[numerical_columns]
data_numerical = (data_numerical - data_numerical.mean()) / data_numerical.std()
data_numerical.describe()

#делаем новую таблицу с переделанными данными

data_train = pd.concat((data_numerical, data_train[binary_columns], data_nonbinary), axis=1)
data_train = pd.DataFrame(data_train, dtype=int)

In [16]:
X = data_train.drop(('Occupancy'), axis=1)  # Выбрасываем столбец 'SalePrice'.
y = data_train['Occupancy']
feature_names = X.columns

#метод главных компонент

pca = PCA(n_components = 5)
XPCAreduced = pca.fit_transform(X)

#обработка данных на тренировочную и тестовую выборку

X_train, X_test, y_train, y_test = train_test_split(XPCAreduced, y, test_size = 0.2, random_state = 11)

N_train, _ = X_train.shape 
N_test,  _ = X_test.shape 


#реализация библиотечной регрессии

lr = LogisticRegression()
lr.fit(X_train, y_train)

y_train_predict = lr.predict(X_train)
y_test_predict = lr.predict(X_test)


print("Модель из sklearn")
print("MAE(Средний модуль ошибки): ", mae(y_test, y_test_predict))
print("Точность: ", round(accuracy_score(y_train, y_train_predict), 3))

Модель из sklearn
MAE(Средний модуль ошибки):  0.012155591572123177
Точность:  0.986


In [17]:
#вручную

#предсказать
def predict_outcome(feature_matrix, weights):
    weights=np.array(weights)
    predictions = np.dot(feature_matrix, weights)
    return predictions

#ошибки
def errors(output,predictions):
    errors=predictions-output
    return errors

#производная
def feature_derivative(errors, feature):
    derivative=np.dot(2,np.dot(feature,errors))
    return derivative
#градиентный спуск
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False
    #Начальные веса -> массив numpy
    weights = np.array(initial_weights)
    while not converged:
        # вычислить прогнозы
        predictions=predict_outcome(feature_matrix,weights)
        # вычислить ошибки
        error=errors(output,predictions)
        gradient_sum_squares = 0 # инициализирование градиента
        # пока не сходится, обновлять каждый вес отдельно:
        for i in range(len(weights)):
            #вызов feature_matrix[:, i] если столбец фич связан с весами[i]
            feature=feature_matrix[:, i]
            deriv=feature_derivative(error,feature)
            #квадратная производная + градиент
            gradient_sum_squares=gradient_sum_squares+(deriv**2)
            # обновить вес
            weights[i]=weights[i] - np.dot(step_size,deriv)

        gradient_magnitude = math.sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
    return(weights)

simple_feature_matrix = XPCAreduced #простая матрица харастеристик
output = y
initial_weights = np.array([0.1, 0.001, 0.001, 0.001, 0.001])
step_size = 0.00001
tolerance = 2.5e7 #допустимое отклонение
simple_weights = regression_gradient_descent(simple_feature_matrix,output, initial_weights, step_size, tolerance)

hp = np.dot(X_train, simple_weights)
#сигмоида (её значение и есть предсказание)
def sigmoidfun(x):
    return 1 / (1 + np.exp(-x))

hand_y_train_predict = np.apply_along_axis(sigmoidfun, 0, hp)

hand_y_train_predict = list(map(lambda x: 1 if x > 0.5 else 0, hand_y_train_predict))

print("Вручную")
print("MAE: ", mae(y_test, y_test_predict))
print("Точность: ", round(accuracy_score(y_train, hand_y_train_predict, 3)))

Вручную
MAE:  0.012155591572123177
Точность:  1.0
